In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from fastai.vision.all import *

In [2]:
base_data_dir = '/kaggle/input/landmark-recognition-2021/'
train_dir = 'train'
test_dir = 'test'
output_dir = '/kaggle/working/'
output_model_dir = '../input/notebook4a5cf7f230/models'
model_name_prefix = 'landmark-recognition-2021_res50'
model_timstamp = '20210905154054'

In [3]:
learner = load_learner(Path(f"{output_model_dir}/{model_name_prefix}_{model_timstamp}.pkl"))

In [4]:
def add_file_path(df, train_test_dir, return_order = ['id','path','landmark_id']):
    df['path'] = df['id'].apply(lambda x:train_test_dir + '/' +\
                                '/'.join(x[0:3]) + '/' + x + '.jpg')
    df['landmark_id'] = df['landmark_id'].astype(str)
    return df[return_order]

def get_predicted_data(learner, test_df, cat_list):
    test_dl = learner.dls.test_dl(test_df)
    prob_tensor, _, pred_tensor = learner.get_preds(dl = test_dl, with_decoded=True)
    pred_list = [f"{cat_list[pred_idx]} {prob_idx[pred_idx]}" 
                 for pred_idx, prob_idx in zip(pred_tensor,prob_tensor)]
    
    test_df['landmarks'] = pred_list
    return test_df
    

def get_inference_df(root_test_dir):
    path_list=[]
    for dirname, _, filenames in os.walk(root_test_dir):
        for filename in filenames:
            path_list.append((filename.split('.')[0],'0'))
#         if len(path_list)>0: break
    return pd.DataFrame(path_list, columns=['id','landmark_id'])
    

In [5]:
test_df = get_inference_df(base_data_dir + '/' + test_dir+ '/')
test_df = add_file_path(test_df, test_dir)
test_df.head()

,id,path,landmark_id
0,777f9efff0fc6b81,test/7/7/7/777f9efff0fc6b81.jpg,0
1,7774e44062fbd8bc,test/7/7/7/7774e44062fbd8bc.jpg,0
2,777173e839e6cfa7,test/7/7/7/777173e839e6cfa7.jpg,0
3,7726658184c7e337,test/7/7/2/7726658184c7e337.jpg,0
4,77be72c73bbf4f18,test/7/7/b/77be72c73bbf4f18.jpg,0


In [6]:
cat_list = learner.dls.vocab
cat_list

['10419', '113209', '120734', '126637', '138982', '139894', '149980', '161902', '168098', '176018', '176528', '177870', '1924', '194914', '20120', '20409', '36748', '46705', '83144', '9070']

In [7]:
pred_df = get_predicted_data(learner, test_df, cat_list)
pred_df.head()

,id,path,landmark_id,landmarks
0,777f9efff0fc6b81,test/7/7/7/777f9efff0fc6b81.jpg,0,20120 0.7920076847076416
1,7774e44062fbd8bc,test/7/7/7/7774e44062fbd8bc.jpg,0,176528 0.8594590425491333
2,777173e839e6cfa7,test/7/7/7/777173e839e6cfa7.jpg,0,176528 0.8384840488433838
3,7726658184c7e337,test/7/7/2/7726658184c7e337.jpg,0,161902 0.8116684556007385
4,77be72c73bbf4f18,test/7/7/b/77be72c73bbf4f18.jpg,0,138982 0.9177926182746887


In [8]:
pred_df[['id','landmarks']].to_csv(f"{output_dir}/submission.csv",
               index= False)